## Problem

Detecting whether an article is Fake News using Stance Detection. Stance detection takes a headline and body text of an article and classifies the stance of the body text relative to headline. 

In [2]:
import pandas as pd 

import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
bodies = pd.read_csv('./Data_Fake_News /train_bodies.csv')
stances = pd.read_csv('./Data_Fake_News /train_stances.csv')


In [29]:
stances['Stance'].value_counts(normalize=True)

unrelated    0.731310
discuss      0.178280
agree        0.073601
disagree     0.016809
Name: Stance, dtype: float64

## EDA

In [5]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [6]:
stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [7]:
stances[stances['Body ID'] == 712]

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1787,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss
3974,Mexico police find mass grave near site 43 stu...,712,unrelated
4936,Mexico Says Missing Students Not Found In Firs...,712,unrelated
5210,New iOS 8 bug can delete all of your iCloud do...,712,unrelated
5863,Return of the Mac: Seth Rogen in talks to star...,712,discuss
6199,Seth Rogen Is Woz,712,discuss
6756,Mexico finds 4 more graves at site of suspecte...,712,unrelated
7526,Are missing students in mass graves found near...,712,unrelated
9003,Mexico prosecutor: Students not in 1st mass gr...,712,unrelated


In [8]:
stances['Body ID'].value_counts()

1921    187
1948    175
40      172
524     171
1549    166
304     154
1385    151
125     145
2367    143
220     141
1438    141
195     140
2296    136
35      131
1786    131
1883    131
2520    127
1034    127
2252    126
1574    125
2307    125
527     125
2175    124
1627    123
2404    123
1289    122
2115    121
2096    120
1040    118
1893    117
       ... 
907       1
370       1
210       1
146       1
114       1
1542      1
63        1
76        1
390       1
515       1
193       1
464       1
355       1
323       1
624       1
282       1
18        1
797       1
701       1
362       1
2311      1
6         1
915       1
70        1
151       1
376       1
140       1
307       1
1066      1
59        1
Name: Body ID, Length: 1683, dtype: int64

In [9]:
stances['Stance'].unique()

array(['unrelated', 'agree', 'disagree', 'discuss'], dtype=object)

In [10]:
print('Bodies shape:', bodies.shape)
print('Stances shape:', stances.shape)


Bodies shape: (1683, 2)
Stances shape: (49972, 3)


In [11]:
stances['Stance'].value_counts()/stances.shape[0]

unrelated    0.731310
discuss      0.178280
agree        0.073601
disagree     0.016809
Name: Stance, dtype: float64

In [12]:
#why are stances rows much larger

In [13]:
bodies.isnull().sum()

Body ID        0
articleBody    0
dtype: int64

In [14]:
stances.isnull().sum()

Headline    0
Body ID     0
Stance      0
dtype: int64

In [15]:
bodies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683 entries, 0 to 1682
Data columns (total 2 columns):
Body ID        1683 non-null int64
articleBody    1683 non-null object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [16]:
stances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49972 entries, 0 to 49971
Data columns (total 3 columns):
Headline    49972 non-null object
Body ID     49972 non-null int64
Stance      49972 non-null object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
print("Total stances: " + str(len(stances.Stance)))
print("Total article bodies: " + str(len(bodies.articleBody)))

Total stances: 49972
Total article bodies: 1683


In [18]:
#merge on bodyID 

df_all = stances.merge(bodies, on = 'Body ID')
df_all = df_all.drop('Stance', axis =1)

In [19]:
df_all.isnull().sum()

Headline       0
Body ID        0
articleBody    0
dtype: int64

In [20]:
df_all.shape

(49972, 3)

## NLP on Body

In [21]:
# A:
X_train = df_all[['articleBody', 'Headline']]
y_train = stances['Stance']
X_test = df_all[['articleBody', 'Headline']]
y_test = stances['Stance']

In [143]:
X_train.shape

(49972, 2)

In [148]:
cvec_body = CountVectorizer(stop_words = 'english')

In [149]:
cvec_headline = CountVectorizer(stop_words = 'english')

In [150]:
train_body_cvec = cvec_body.fit_transform(X_train['articleBody'])
test_body_cvec = cvec.transform(X_test['articleBody'])

In [151]:
train_body_cvec.shape

(49972, 22888)

In [163]:
train_body_array = train_body_cvec.toarray()
train_body_array.shape

(49972, 3167)

In [171]:
train_headline_cvec = cvec_body.fit_transform(X_train['Headline'])
train_headline_array = train_headline_cvec.toarray()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [155]:
train_headline_cvec.shape

(49972, 3167)

In [168]:
test_headline_array = test_headline_cvec.toarray()
test_body_array = test_body_cvec.toarray()

In [169]:
X_train = np.concatenate((train_headline_array, train_body_array))
y_train = np.concatenate((test_headline_array, test_body_array))

In [22]:
X_train = df_all[['articleBody']]
y_train = stances['Stance']
X_test = df_all[['articleBody']]
y_test = stances['Stance']

In [23]:
cvec_body = CountVectorizer(stop_words = 'english')

In [27]:
train_body_cvec = cvec_body.fit_transform(X_train['articleBody'])
test_body_cvec = cvec_body.transform(X_test['articleBody'])

In [28]:
lr = LogisticRegression()
lr.fit(train_body_cvec, y_train)
lr.score(test_body_cvec, y_test)

0.7320899703834147

In [ ]:
combine headline and body cvec and that will be x_train
fit model on X_train 
if model doesnt do well do truncated svd then cosine similarity between bodies and headline which will become a feature 


X_train_fit.shape

In [123]:
X_train_fit.shape

(2, 2)

In [124]:
X_test_count.todense()

matrix([[1, 0],
        [0, 1]])

In [125]:
cvec.get_feature_names()

['articlebody', 'headline']

In [126]:
lr = LogisticRegression()
lr.fit(X_train_fit, y_train)
lr.score(X_test_count, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [2, 49972]

In [116]:
df  = pd.DataFrame(X_test_count.todense(),
                   columns=cvec.get_feature_names(), index = [bodies.articleBody]) 
df.head()


ValueError: Shape of passed values is (2, 2), indices imply (2, 1683)

In [56]:
df.sort_index(axis = 0, ascending=False).head()

00  000  000000  000ft  \
articleBody                                                                  
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn            0    0       0      0   
• Jihadi John named by Washington Post as Moham...   1    0       0      0   
“Star Wars: The Force Awakens” is one of the mo...   0    0       0      0   
“A Message To America,” a video uploaded Tuesda...   0    0       0      0   
via Daily Buzz Live\n\nStrange natural occurren...   0    0       0      0   

                                                    000km  00144feab7de  005  \
articleBody                                                                    
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn               0             0    0   
• Jihadi John named by Washington Post as Moham...      0             0    0   
“Star Wars: The Force Awakens” is one of the mo...      0             0    0   
“A Message To America,” a video uploaded Tuesda...      0             0    0   
via Daily Buzz Live\n\nStrange natural occurren...      0             0    0   

                                                    0054321535  005930  009  \
articleBody                                                                   
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn                    0       0    0   
• Jihadi John named by Washington Post as Moham...           0       0    0   
“Star Wars: The Force Awakens” is one of the mo...           0       0    0   
“A Message To America,” a video uploaded Tuesda...           0       0    0   
via Daily Buzz Live\n\nStrange natural occurren...           0       0    0   

                                                   ...  يحرق  يشارك  يفصل  \
articleBody                                        ...                      
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn          ...     0      0     0   
• Jihadi John named by Washington Post as Moham... ...     0      0     0   
“Star Wars: The Force Awakens” is one of the mo... ...     0      0     0   
“A Message To America,” a video uploaded Tuesda... ...     0      0     0   
via Daily Buzz Live\n\nStrange natural occurren... ...     0      0     0   

                                                    يوسف  ٢٣  刷榜  您所访问的资源已不存在  \
articleBody                                                                     
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn              0   0   0            1   
• Jihadi John named by Washington Post as Moham...     0   0   0            0   
“Star Wars: The Force Awakens” is one of the mo...     0   0   0            0   
“A Message To America,” a video uploaded Tuesda...     0   0   0            0   
via Daily Buzz Live\n\nStrange natural occurren...     0   0   0            0   

                                                    查理  查看更多请返回网站主页  타미  
articleBody                                                              
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn            0            1   0  
• Jihadi John named by Washington Post as Moham...   0            0   0  
“Star Wars: The Force Awakens” is one of the mo...   0            0   0  
“A Message To America,” a video uploaded Tuesda...   0            0   0  
via Daily Buzz Live\n\nStrange natural occurren...   0            0   0  

[5 rows x 22888 columns]

In [57]:
df.sum().sort_values(ascending=False).head(10)

said      4232
apple     1790
watch     1152
video     1145
state     1137
people    1131
told      1130
new       1121
isis      1052
news      1048
dtype: int64

In [58]:
df = df.reindex_axis(df.sum().sort_values(ascending=False).index, axis =1)
df

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if __name__ == '__main__':


said  apple  watch  video  \
articleBody                                                                     
A small meteorite crashed into a wooded area in...     9      0      0      0   
Last week we hinted at what was to come as Ebol...     0      0      0      0   
(NEWSER) – Wonder how long a Quarter Pounder wi...     0      0      0      0   
Posting photos of a gun-toting child online, IS...     8      0      0      0   
At least 25 suspected Boko Haram insurgents wer...    10      0      0      0   
There is so much fake stuff on the Internet in ...     4      0      0      4   
(CNN) -- A meteorite crashed down in Managua, N...     3      0      0      0   
Move over, Netflix and Hulu.\nWord has it that ...     3      0      0      8   
We’ve all seen the traditional depictions of Go...     3      0      0      0   
A SOLDIER has been shot at Canada’s National Wa...     0      0      0      0   
mboxCreate('FoxNews-Politics-Autoplay-Videos-In...     9      0      0      4   
Don't fucking cheat on Cassy, aka @NessLovnTrey...     0      0      0      2   
Kai the shar pei-crossbreed was discovered tied...     2      0      0      0   
An article saying NASA confirmed six days of “t...     0      0      0      0   
Italian fisherman Dino Ferrari landed what coul...     0      0      0      0   
HBO's subscription streaming service will be ca...     0      5      1      0   
In a sprawling Facebook post and subsequent int...     1      0      0      0   
Macaulay Culkin is not dead. The actor is just ...     0      0      0      0   
DUBAI - A prominent Saudi Arabian cleric has wh...     1      0      0      0   
Eran Cicurel, an editor at Voice of Israel, has...     0      0      0      1   
Phoenix, AZ — A Phoenix boy is behind bars toni...     2      0      1      1   
Abdel-Majed Abdel Bary is a former West London ...     1      0      0      1   
DNA tests confirm Lebanon is holding the young ...     5      0      0      0   
FERGUSON, St. Louis (CNN) — Could a newly relea...    12      0      0      1   
Abdel-Majed Abdel Bary, who went by the rap nam...    10      0      0      1   
New Delhi: AK Verma, an executive engineer at t...     2      0      0      0   
The London Metropolitan Police has denied repor...     1      0      0      0   
Young North Korean dictator Kim Jong Un’s healt...     6      0      0      0   
Although Apple has given us our first peek at t...     1     11      9      0   
The small town of Purdon, Texas has been quaran...     0      0      0      0   
...                                                  ...    ...    ...    ...   
Offers have been pouring in to re-home a dog fo...     2      0      0      0   
Although Apple has given us our first peek at t...     1     11      9      0   
Sorry to disappoint, fans of Led Zeppelin, but ...     0      0      0      0   
While other global leaders rant their exasperat...     9      0      0      1   
SAN SEVERINO - Go to the hospital accusing a te...     0      0      0      0   
Islamic State militants appear to have killed m...     0      0      0      6   
Warning graphic image: Oliver Ilic, 22, had to ...     2      0      1      1   
If 9to5Mac’s latest findings are true, than App...     0      5      1      0   
In case you missed it, Vogue Magazine, one of t...     1      0      1      0   
In response to Ebola Scare in Kansas City :\nVi...     2      0      0      0   
News that the ‘Home Alone’ star has allegedly d...     1      0      0      0   
Seven girls aged between 13 and 14 have fallen ...     2      0      0      1   
Fidel Castro was pronounced dead on Twitter som...     2      0      0      0   
28 girls, aged 13 and 14, from a small town wen...     4      0      0      0   
BEIRUT — Islamic State group fighters seized at...     7      0      0      3   
The weather man was reporting from Sugar Mounta...     3      0      0      1   
So… Rebecca Schoenkopf over at Wonkette is pret...     0      0      1      3   
Et

In [61]:
lr = LogisticRegression()
lr.fit(X_train_fit, y_train)
lr.score(X_test_count, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [1683, 49972]

In [63]:
X_train_fit.shape, X_test_count.shape

((1683, 22888), (1683, 22888))

In [64]:
y_train.shape, y_test.shape

((49972,), (49972,))

# Hashing Vectorizer

In [29]:
# A:
hvec = HashingVectorizer(stop_words = 'english')
# Fit on training data
X_train_count = hvec.fit_transform(X_train)

# Don't fit on test data
X_test_count = hvec.transform(X_test)

In [30]:
#1048476 words with stop words 
X_train_count.shape

(1683, 1048576)

In [31]:
df_hv  = pd.DataFrame(hvec.transform(X_test).todense(), index=[bodies.articleBody])  
df_hv.head()

,0,1,2,3,4,5,6,7,8,9,...,1048566,1048567,1048568,1048569,1048570,1048571,1048572,1048573,1048574,1048575
articleBody,,,,,,,,,,,,,,,,,,,,,
"A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a ""relatively small"" meteorite that ""appears to have come off an asteroid that was passing close to Earth."" House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet, said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumberto Garcia, of the Astronomy Center at the National Autonomous University of Nicaragua, said the meteorite could be related to an asteroid that was forecast to pass by the planet Saturday night.\n\n""We have to study it more because it could be ice or rock,"" he said.\n\nWilfried Strauch, an adviser to the Institute of Territorial Studies, said it was ""very strange that no one reported a streak of light. We have to ask if anyone has a photo or something.""\n\nLocal residents reported hearing a loud boom Saturday night, but said they didn't see anything strange in the sky.\n\n""I was sitting on my porch and I saw nothing, then all of a sudden I heard a large blast. We thought it was a bomb because we felt an expansive wave,"" Jorge Santamaria told The Associated Press.\n\nThe site of the crater is near Managua's international airport and an air force base. Only journalists from state media were allowed to visit it.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Last week we hinted at what was to come as Ebola fears spread across America. Today, we get confirmation. As The Daily Caller reports, one passenger at Dulles International Airport outside Washington, D.C. is apparently not taking any chances. A female passenger dressed in a hazmat suit - complete with a full body gown, mask and gloves - was spotted Wednesday waiting for a flight at the airport.\n\n \n\n\n\nSource: The Daily Caller\n\nWe particularly liked the JCPenney bag - maybe that's a new business line for the bankrupt retailer...\n\n* * *\n\nOn a side note, try Halloween stores if you need a Haz-Mat suit in a hurry...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(NEWSER) – Wonder how long a Quarter Pounder with cheese can last? Two Australians say they bought a few McDonald's burgers for friends back in 1995, when they were teens, and one of the friends never showed up. So the kid's burger went uneaten—and stayed that way, Australia's News Network reports. ""We’re pretty sure it’s the oldest burger in the world,"" says one of the men, Casey Dean. Holding onto the burger for their friend ""started off as a joke,"" he adds, but ""the months became years and now, 20 years later, it looks the same as it did the day we bought it, perfectly preserved in its original wrapping.""\n\nDean and his burger-buying mate, Eduard Nitz, even took the burger on Australian TV show The Project last night and ""showed off the mold-free specimen,"" News 9 reports. The pair offered to take a bite of it for charity but were dissuaded by the show's hosts. They've also started a Facebook page for the burger called ""Can This 20 Year Old Burger Get More Likes Than Kanye West?"" that has more than 4,044 likes as of this writing. And they're selling an iTunes song, ""Free the Burger,"" for $1.69, 

In [32]:
df_hv.sum().sort_values(ascending=False).head(10)

982957    181.456057
452752     76.557963
705940     55.043240
864        49.308233
681351     48.170896
26634      47.116314
661528     47.031982
360502     45.700292
204878     42.528627
637628     38.351424
dtype: float64

In [33]:
df_hv = df_hv.reindex_axis(df_hv.sum().sort_values(ascending=False).index, axis =1)
df_hv

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if __name__ == '__main__':


982957    452752  \
articleBody                                                              
A small meteorite crashed into a wooded area in...  0.478345  0.000000   
Last week we hinted at what was to come as Ebol...  0.000000  0.000000   
(NEWSER) – Wonder how long a Quarter Pounder wi...  0.000000  0.000000   
Posting photos of a gun-toting child online, IS...  0.281265  0.000000   
At least 25 suspected Boko Haram insurgents wer...  0.428746  0.000000   
There is so much fake stuff on the Internet in ...  0.104577  0.000000   
(CNN) -- A meteorite crashed down in Managua, N...  0.120873  0.000000   
Move over, Netflix and Hulu.\nWord has it that ...  0.091457  0.000000   
We’ve all seen the traditional depictions of Go...  0.191663  0.000000   
A SOLDIER has been shot at Canada’s National Wa...  0.000000  0.000000   
mboxCreate('FoxNews-Politics-Autoplay-Videos-In...  0.326464  0.000000   
Don't fucking cheat on Cassy, aka @NessLovnTrey...  0.000000  0.000000   
Kai the shar pei-crossbreed was discovered tied...  0.107058  0.000000   
An article saying NASA confirmed six days of “t...  0.000000  0.000000   
Italian fisherman Dino Ferrari landed what coul...  0.000000  0.000000   
HBO's subscription streaming service will be ca...  0.000000  0.168838   
In a sprawling Facebook post and subsequent int...  0.049690  0.000000   
Macaulay Culkin is not dead. The actor is just ...  0.000000  0.000000   
DUBAI - A prominent Saudi Arabian cleric has wh...  0.062378  0.000000   
Eran Cicurel, an editor at Voice of Israel, has...  0.000000  0.000000   
Phoenix, AZ — A Phoenix boy is behind bars toni...  0.055427  0.000000   
Abdel-Majed Abdel Bary is a former West London ...  0.055815  0.000000   
DNA tests confirm Lebanon is holding the young ...  0.222497  0.000000   
FERGUSON, St. Louis (CNN) — Could a newly relea...  0.367538  0.000000   
Abdel-Majed Abdel Bary, who went by the rap nam...  0.233126  0.000000   
New Delhi: AK Verma, an executive engineer at t...  0.138013  0.000000   
The London Metropolitan Police has denied repor...  0.154303  0.000000   
Young North Korean dictator Kim Jong Un’s healt...  0.283473  0.000000   
Although Apple has given us our first peek at t...  0.049088  0.539969   
The small town of Purdon, Texas has been quaran...  0.000000  0.000000   
...                                                      ...       ...   
Offers have been pouring in to re-home a dog fo...  0.088736  0.000000   
Although Apple has given us our first peek at t...  0.049088  0.539969   
Sorry to disappoint, fans of Led Zeppelin, but ...  0.000000  0.000000   
While other global leaders rant their exasperat...  0.333562  0.000000   
SAN SEVERINO - Go to the hospital accusing a te...  0.000000  0.000000   
Islamic State militants appear to have killed m...  0.000000  0.000000   
Warning graphic image: Oliver Ilic, 22, had to ...  0.119098  0.000000   
If 9to5Mac’s latest findings are true, than App...  0.000000  0.171701   
In case you missed it, Vogue Magazine, one of t...  0.044811  0.000000   
In response to Ebola Scare in Kansas City :\nVi...  0.180334  0.000000   
News that the ‘Home Alone’ star has allegedly d...  0.044721  0.000000   
Seven girls aged between 13 and 14 have fallen ...  0.095130  0.000000   
Fidel Castro was pronounced dead on Twitter som...  0.105409  0.000000   
28 girls, aged 13 and 14, from a small town wen...  0.274721  0.000000   
BEIRUT — Islamic State group fighters seized at...  0.299298  0.000000   
The weather man was reporting from Sugar Mounta...  0.182237  0.000000   
So… Rebecca Schoenkopf over at Wonkette is pret...  0.000000  0.000000   
Et si Dieu était une femme? C'est ce qu'affirme...  0.000000  0.000000   
On Friday, a rumor cropped up that one of the n...  0.142494  0.000000   
Anna Wintour, editor-in-chief on Vogue magazine...  0.145180  0.000000   
The hoax story has been circulating far and wid...  0.000000  0.000000   
THE hunt is on to find the owner of a dog who w...  0.2433

In [ ]:
#what can i do with this 

# TF IDF 

In [36]:
tvec = TfidfVectorizer(stop_words = 'english')

X_train_count = tvec.fit_transform(X_train)

X_test_count = tvec.transform(X_test)

In [37]:
X_train_count.shape

(1683, 22888)

In [43]:
df_tfidf  = pd.DataFrame(tvec.transform(X_test).todense(),
                   columns=tvec.get_feature_names(),
                   index=[bodies.articleBody])
df_tfidf.head()
#df.transpose().sort_values('bodies.articleBody', ascending=False).head(10).transpose()

,00,000,000000,000ft,000km,00144feab7de,005,0054321535,005930,009,...,يحرق,يشارك,يفصل,يوسف,٢٣,刷榜,您所访问的资源已不存在,查理,查看更多请返回网站主页,타미
articleBody,,,,,,,,,,,,,,,,,,,,,
"A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a ""relatively small"" meteorite that ""appears to have come off an asteroid that was passing close to Earth."" House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet, said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumberto Garcia, of the Astronomy Center at the National Autonomous University of Nicaragua, said the meteorite could be related to an asteroid that was forecast to pass by the planet Saturday night.\n\n""We have to study it more because it could be ice or rock,"" he said.\n\nWilfried Strauch, an adviser to the Institute of Territorial Studies, said it was ""very strange that no one reported a streak of light. We have to ask if anyone has a photo or something.""\n\nLocal residents reported hearing a loud boom Saturday night, but said they didn't see anything strange in the sky.\n\n""I was sitting on my porch and I saw nothing, then all of a sudden I heard a large blast. We thought it was a bomb because we felt an expansive wave,"" Jorge Santamaria told The Associated Press.\n\nThe site of the crater is near Managua's international airport and an air force base. Only journalists from state media were allowed to visit it.",0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Last week we hinted at what was to come as Ebola fears spread across America. Today, we get confirmation. As The Daily Caller reports, one passenger at Dulles International Airport outside Washington, D.C. is apparently not taking any chances. A female passenger dressed in a hazmat suit - complete with a full body gown, mask and gloves - was spotted Wednesday waiting for a flight at the airport.\n\n \n\n\n\nSource: The Daily Caller\n\nWe particularly liked the JCPenney bag - maybe that's a new business line for the bankrupt retailer...\n\n* * *\n\nOn a side note, try Halloween stores if you need a Haz-Mat suit in a hurry...",0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(NEWSER) – Wonder how long a Quarter Pounder with cheese can last? Two Australians say they bought a few McDonald's burgers for friends back in 1995, when they were teens, and one of the friends never showed up. So the kid's burger went uneaten—and stayed that way, Australia's News Network reports. ""We’re pretty sure it’s the oldest burger in the world,"" says one of the men, Casey Dean. Holding onto the burger for their friend ""started off as a joke,"" he adds, but ""the months became years and now, 20 years later, it looks the same as it did the day we bought it, perfectly preserved in its original wrapping.""\n\nDean and his burger-buying mate, Eduard Nitz, even took the burger on Australian TV show The Project last night and ""showed off the mold-free specimen,"" News 9 reports. The pair offered to take a bite of it for charity but were dissuaded by the show's hosts. They've also started a Facebook page for the burger called ""Can This 20 Year Old Burger Get More Likes Than Kanye West?"" that has more than 4,044 likes as of this writing. And they're selling an iTunes song, ""

In [44]:
df_tfidf.sort_index(axis = 0, ascending=False).head()

00  000  000000  \
articleBody                                                                 
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn           0.000000  0.0     0.0   
• Jihadi John named by Washington Post as Moham...  0.026489  0.0     0.0   
“Star Wars: The Force Awakens” is one of the mo...  0.000000  0.0     0.0   
“A Message To America,” a video uploaded Tuesda...  0.000000  0.0     0.0   
via Daily Buzz Live\n\nStrange natural occurren...  0.000000  0.0     0.0   

                                                    000ft  000km  \
articleBody                                                        
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn             0.0    0.0   
• Jihadi John named by Washington Post as Moham...    0.0    0.0   
“Star Wars: The Force Awakens” is one of the mo...    0.0    0.0   
“A Message To America,” a video uploaded Tuesda...    0.0    0.0   
via Daily Buzz Live\n\nStrange natural occurren...    0.0    0.0   

                                                    00144feab7de  005  \
articleBody                                                             
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn                    0.0  0.0   
• Jihadi John named by Washington Post as Moham...           0.0  0.0   
“Star Wars: The Force Awakens” is one of the mo...           0.0  0.0   
“A Message To America,” a video uploaded Tuesda...           0.0  0.0   
via Daily Buzz Live\n\nStrange natural occurren...           0.0  0.0   

                                                    0054321535  005930  009  \
articleBody                                                                   
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn                  0.0     0.0  0.0   
• Jihadi John named by Washington Post as Moham...         0.0     0.0  0.0   
“Star Wars: The Force Awakens” is one of the mo...         0.0     0.0  0.0   
“A Message To America,” a video uploaded Tuesda...         0.0     0.0  0.0   
via Daily Buzz Live\n\nStrange natural occurren...         0.0     0.0  0.0   

                                                   ...   يحرق  يشارك  يفصل  \
articleBody                                        ...                       
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn          ...    0.0    0.0   0.0   
• Jihadi John named by Washington Post as Moham... ...    0.0    0.0   0.0   
“Star Wars: The Force Awakens” is one of the mo... ...    0.0    0.0   0.0   
“A Message To America,” a video uploaded Tuesda... ...    0.0    0.0   0.0   
via Daily Buzz Live\n\nStrange natural occurren... ...    0.0    0.0   0.0   

                                                    يوسف   ٢٣   刷榜  \
articleBody                                                          
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn            0.0  0.0  0.0   
• Jihadi John named by Washington Post as Moham...   0.0  0.0  0.0   
“Star Wars: The Force Awakens” is one of the mo...   0.0  0.0  0.0   
“A Message To America,” a video uploaded Tuesda...   0.0  0.0  0.0   
via Daily Buzz Live\n\nStrange natural occurren...   0.0  0.0  0.0   

                                                    您所访问的资源已不存在   查理  \
articleBody                                                            
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn              0.522877  0.0   
• Jihadi John named by Washington Post as Moham...     0.000000  0.0   
“Star Wars: The Force Awakens” is one of the mo...     0.000000  0.0   
“A Message To America,” a video uploaded Tuesda...     0.000000  0.0   
via Daily Buzz Live\n\nStrange natural occurren...     0.000000  0.0   

                                                    查看更多请返回网站主页   타미  
articleBody                                                           
您所访问的资源已不存在。\n查看更多请返回网站主页。\n» cctvnews.cn              0.522877  0.0  
• Jihadi John named by Washington Post as Moham...     0.000000  0.0  
“Star Wars: The Force Awakens” is one of the mo...     0.000000  0.0  
“A Message To America,” a video uploaded Tuesda...     0.000000  0.0  
via Da

In [45]:
df_tfidf.sum().sort_values(ascending=False).head(10)

said     58.470804
apple    56.272225
watch    35.504379
isis     33.908509
video    29.498077
man      25.442050
state    25.371016
al       24.978154
news     24.050054
kim      23.770770
dtype: float64

In [46]:
df_tfidf = df_tfidf.reindex_axis(df_tfidf.sum().sort_values(ascending=False).index, axis =1)
df_tfidf

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if __name__ == '__main__':


said     apple  \
articleBody                                                              
A small meteorite crashed into a wooded area in...  0.169408  0.000000   
Last week we hinted at what was to come as Ebol...  0.000000  0.000000   
(NEWSER) – Wonder how long a Quarter Pounder wi...  0.000000  0.000000   
Posting photos of a gun-toting child online, IS...  0.106499  0.000000   
At least 25 suspected Boko Haram insurgents wer...  0.143387  0.000000   
There is so much fake stuff on the Internet in ...  0.032597  0.000000   
(CNN) -- A meteorite crashed down in Managua, N...  0.035991  0.000000   
Move over, Netflix and Hulu.\nWord has it that ...  0.027354  0.000000   
We’ve all seen the traditional depictions of Go...  0.059858  0.000000   
A SOLDIER has been shot at Canada’s National Wa...  0.000000  0.000000   
mboxCreate('FoxNews-Politics-Autoplay-Videos-In...  0.125790  0.000000   
Don't fucking cheat on Cassy, aka @NessLovnTrey...  0.000000  0.000000   
Kai the shar pei-crossbreed was discovered tied...  0.029725  0.000000   
An article saying NASA confirmed six days of “t...  0.000000  0.000000   
Italian fisherman Dino Ferrari landed what coul...  0.000000  0.000000   
HBO's subscription streaming service will be ca...  0.000000  0.098926   
In a sprawling Facebook post and subsequent int...  0.015043  0.000000   
Macaulay Culkin is not dead. The actor is just ...  0.000000  0.000000   
DUBAI - A prominent Saudi Arabian cleric has wh...  0.018490  0.000000   
Eran Cicurel, an editor at Voice of Israel, has...  0.000000  0.000000   
Phoenix, AZ — A Phoenix boy is behind bars toni...  0.014897  0.000000   
Abdel-Majed Abdel Bary is a former West London ...  0.017965  0.000000   
DNA tests confirm Lebanon is holding the young ...  0.073759  0.000000   
FERGUSON, St. Louis (CNN) — Could a newly relea...  0.131038  0.000000   
Abdel-Majed Abdel Bary, who went by the rap nam...  0.088267  0.000000   
New Delhi: AK Verma, an executive engineer at t...  0.042920  0.000000   
The London Metropolitan Police has denied repor...  0.055221  0.000000   
Young North Korean dictator Kim Jong Un’s healt...  0.100593  0.000000   
Although Apple has given us our first peek at t...  0.018136  0.430345   
The small town of Purdon, Texas has been quaran...  0.000000  0.000000   
...                                                      ...       ...   
Offers have been pouring in to re-home a dog fo...  0.023927  0.000000   
Although Apple has given us our first peek at t...  0.018136  0.430345   
Sorry to disappoint, fans of Led Zeppelin, but ...  0.000000  0.000000   
While other global leaders rant their exasperat...  0.106769  0.000000   
SAN SEVERINO - Go to the hospital accusing a te...  0.000000  0.000000   
Islamic State militants appear to have killed m...  0.000000  0.000000   
Warning graphic image: Oliver Ilic, 22, had to ...  0.036308  0.000000   
If 9to5Mac’s latest findings are true, than App...  0.000000  0.118410   
In case you missed it, Vogue Magazine, one of t...  0.012915  0.000000   
In response to Ebola Scare in Kansas City :\nVi...  0.059055  0.000000   
News that the ‘Home Alone’ star has allegedly d...  0.014590  0.000000   
Seven girls aged between 13 and 14 have fallen ...  0.029475  0.000000   
Fidel Castro was pronounced dead on Twitter som...  0.029901  0.000000   
28 girls, aged 13 and 14, from a small town wen...  0.088758  0.000000   
BEIRUT — Islamic State group fighters seized at...  0.114183  0.000000   
The weather man was reporting from Sugar Mounta...  0.054752  0.000000   
So… Rebecca Schoenkopf over at Wonkette is pret...  0.000000  0.000000   
Et si Dieu était une femme? C'est ce qu'affirme...  0.000000  0.000000   
On Friday, a rumor cropped up that one of the n...  0.043085  0.000000   
Anna Wintour, editor-in-chief on Vogue magazine...  0.044609  0.000000   
The hoax story has been circulating far and wid...  0.000000  0.000000   
THE hunt is on to find the owner of a dog who w...  0.070539

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_count, y_train)
lr.score(X_test_count, y_test)

## NLP on Title 